In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import numpy as np
import matplotlib.pyplot as plt
from DSS_analyzer_Mariner import Data1D_GAUGE, Data1D_PumpingCurve
datapath = "./"
gauge_data_folder = datapath + "s_well/gauge_data/"
gauge_dataframe = Data1D_GAUGE.Data1D_GAUGE(gauge_data_folder + f"gauge6_data_swell.npz")
# Plot the gauge data
from DSS_analyzer_Mariner import Data1D_PumpingCurve
pumping_curve_filepath = datapath + f"prod/pumping_curve/pumping_curve_stage7.npz"
pumping_curve = Data1D_PumpingCurve.Data1D_PumpingCurve(pumping_curve_filepath)

# Crop the pressure gauge data,
start_time = pumping_curve.get_start_time()
end_time = pumping_curve.get_end_time()
gauge_dataframe.crop(start_time, end_time)

gauge_md_index = np.array([265, 520,  789, 1043, 1312, 1567])
frac_hit_index = np.array([589, 830])

In [17]:
# Interpolate the gauge data for any time point, the function would return the interpolated value
from scipy.interpolate import interp1d
def interpolate_gauge_data(time):
    gauge_data = gauge_dataframe.data
    gauge_taxis = gauge_dataframe.calculate_time()
    gauge_interpolator = interp1d(gauge_taxis * 3600, gauge_data, kind='linear')
    return gauge_interpolator(time)

In [18]:
# Simulator parameters
nx = 10000
# nx = 1800
total_time = 13000
d = 140 * np.ones(nx)
dx = 1
x = np.arange(0, nx*dx, dx)
x -= 4000
u_snapshot = [] # store the snapshot of u
u_current = np.zeros(nx)
t = list([0]) # store the time points
dt = 0
dt_init = 10.0

In [19]:
# Load the pumping curve data at next stage
pumping_curve_filepath = datapath + f"prod/pumping_curve/pumping_curve_stage8.npz"
pumping_curve = Data1D_PumpingCurve.Data1D_PumpingCurve(pumping_curve_filepath)

In [20]:
end_time_full = pumping_curve.get_end_time()
end_time_full

datetime.datetime(2020, 3, 18, 14, 39, 26)

In [21]:
print("The total time of 2 stages is", (end_time_full - start_time).total_seconds(), "sec")

The total time of 2 stages is 36898.0 sec
